In [106]:
from collections import Counter
import re
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    # print('Texts:')
    list_t = []
    for text in texts:
        # try:
        #     text = float(text.description))
        # except:
        text = text.description
        list_t.append(text)
        #print('\n"{}"'.format(text.description))

        # vertices = (['({},{})'.format(vertex.x, vertex.y)
        #             for vertex in text.bounding_poly.vertices])

        # print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    return list_t

def check_num(num):
    try:
        float(num)
        return True
    except:
        return False
def replace_invalid(list_detected_text_2):
    f = lambda x: x.replace(" ", ",", ":", ";", "!", "-", "=", "+")
    invalid_chars = [' ', ',', ';', '{', '}', '(', ')', '\n', '\t', '=', ":", ";", "!", "-", "=", "+","/"]
    invalid_table = str.maketrans({k:'' for k in invalid_chars})
    list_valid = [a.translate(invalid_table) for a in list_detected_text_2]
    return list_valid

# def replace_invalid2(list_detected_text_2):
#     f = lambda x: x.replace(" ", ",", ":", ";", "!", "-", "=", "+")
#     invalid_chars = ['.', ',', ';', '{', '}', '(', ')', '\n', '\t', '=', ":", ";", "!", "-", "=", "+", "/"]
#     invalid_table = str.maketrans({k:'.' for k in invalid_chars})
#     list_valid = [a.translate(invalid_table) for a in list_detected_text_2]
#     return list_valid

def islatlong(num):
    try:
        if(float(num) > 10000):
            return True
    except ValueError:
        return False

def islen(num):
    try:
        if(float(num) < 1000) and len(num) > 1:
            return True
    except ValueError:
        return False

def check_point(x,i):
    try:
        x.index('.') == i
        return True
    except: 
        return False
def detect_cordinate(img):
    list_t_f = detect_text(img)
    list_a = re.findall("\d\d\d\d\d\d\d\s\d+", list_t_f[0]) + re.findall("\d\d\d\d\d\d\s\d+", list_t_f[0])
    for a in list_a:
        list_t_f[0] = list_t_f[0].replace(a, a.replace(' ','.'))
    list_t_1 = list_t_f[0].replace(',','.')
    temp_list = list_t_1.split('\n')
    temp_list1 = [a.split(' ') for a in temp_list]
    list_t = sum(temp_list1, [])
    
    list_valid1 = [a for a in list_t if check_num(a[0])]
    list_valid2 = replace_invalid(list_valid1)
    # list_valid2 = replace_invalid2(list_valid2)
    list_fre = [a[:2] for a in list_valid2]
    c = Counter(list_fre)
    list_com = [a[0] for a in c.most_common(2)]
    
    dict_temp = {}
    count = 0
    i= 7
    for a in list_com:
        dict_temp[count] = [re.findall("\d*.\d+", b)[0].replace('-','') for b in list_valid2 if b.startswith(a)]
        temp = []
        for a in dict_temp[count]:
            if check_point(a,i) == True:
                temp.append(a)
            else:
                temp.append(a[:i]+"."+a[i:])
        dict_temp[count] = temp
        i = 6
        count = count + 1
    
    cor = {}
    count = 1
    for i,j in zip(dict_temp[0],dict_temp[1]):
        cor[count] = (float(i),float(j))
        count = count + 1
    
    
    list_len =  [float(a) for a in list_valid2[-len(cor):] if islen(a)]
    
    return cor, list_len

In [109]:
# a,b = detect_cordinate("images/OCR_Internet_Collection_EUS09.jpg")
list_t_f  = detect_text("images/OCR_Internet_Collection_EUS10.jpg")
# list_a = re.findall("\d\d\d\d\d\d\d\s\d+", list_t_f[0]) + re.findall("\d\d\d\d\d\d\s\d+", list_t_f[0])
list_temp = list_t_f[0].split('\n')
list_temp
list_temp2 = []
for a in list_temp:
    if a[0:1].isdigit():
        list_temp2.append(a)
list_temp2
# list_temp2
# detect_text("images/OCR_Internet_Collection_EUS09.jpg")
# for a in list_a:
#         list_t_f[0] = list_t_f[0].replace(a, a.replace(' ','.'))

list_temp3 = []
for a in list_temp2:
    if (" " in a) and (len(a) > 12):
        list_temp3 = list_temp3 + a.split(" ")
    else:
        list_temp3.append(a.replace(" ", "."))
list_temp3

index_point = []
for a in list_temp3:
    try:
        index_point.append(a.index('.'))
    except:
        index_point.append(-1)
max(index_point)

def check_point2(x):
    try:
        x.index('.')
        return True
    except:
        return False
new_list_temp = []   
for l in list_temp3:
    if check_point2(l) == False:
        i = max(index_point)
        if len(l) >= i:
            try:
                new_list_temp.append(str(float(l[:i]+"."+l[i:])))
            except:
                pass
    else:
        try:
            new_list_temp.append(str(float(l)))
        except:
            pass
        

list_valid2 = replace_invalid(new_list_temp)
list_fre = [a[:2] for a in list_valid2]
c = Counter(list_fre)
list_com = [a[0] for a in c.most_common(2)]
list_com.sort()

dict_temp = {}
count = 0
i= 7
for a in list_com:
    dict_temp[count] = [re.findall("\d*.\d+", b)[0].replace('-','') for b in list_valid2 if b.startswith(a)]
    temp = []
    for a in dict_temp[count]:
        if check_point(a,i) == True:
            temp.append(a)
        else:
            temp.append(a[:i]+"."+a[i:])
    dict_temp[count] = temp
    i = 6
    count = count + 1
    
cor = {}
count = 1
for i,j in zip(dict_temp[0],dict_temp[1]):
    cor[count] = (float(i),float(j))
    count = count + 1
list_len =  [float(a) for a in list_valid2[-len(cor):] if islen(a)]

In [114]:
list_valid2

['588787.37',
 '1190671.54',
 '1190669.45',
 '588791.91',
 '1190644.82',
 '588780.57',
 '1190646.82',
 '588775.99',
 '588787.37',
 '5.0',
 '27.11',
 '5.0',
 '27.21']

In [203]:
list_dir = os.listdir("images")
try:
    list_dir.remove('.ipynb_checkpoints')
except:
    pass
list_dir.sort()
list_dir

['OCR_Internet_Collection_EUS01.jpg',
 'OCR_Internet_Collection_EUS02.jpg',
 'OCR_Internet_Collection_EUS03.jpg',
 'OCR_Internet_Collection_EUS04.jpg',
 'OCR_Internet_Collection_EUS05.jpg',
 'OCR_Internet_Collection_EUS06.jpg',
 'OCR_Internet_Collection_EUS07.jpg',
 'OCR_Internet_Collection_EUS08.jpg',
 'OCR_Internet_Collection_EUS09.jpg',
 'OCR_Internet_Collection_EUS10.jpg',
 'OCR_Internet_Collection_EUS11.jpg',
 'OCR_Internet_Collection_EUS12.jpg',
 'OCR_Internet_Collection_EUS13.jpg',
 'OCR_Internet_Collection_EUS14.jpg',
 'OCR_Internet_Collection_EUS15.jpg',
 'OCR_Internet_Collection_EUS16.jpg',
 'OCR_Internet_Collection_EUS17.jpg',
 'OCR_Internet_Collection_EUS18.jpg',
 'OCR_Internet_Collection_EUS19.jpg',
 'OCR_Internet_Collection_EUS20.jpg',
 'OCR_Internet_Collection_EUS21.jpg',
 'OCR_Internet_Collection_EUS22.jpg',
 'OCR_Internet_Collection_EUS23.jpg',
 'OCR_Internet_Collection_EUS24.jpg',
 'OCR_Internet_Collection_EUS25.jpg',
 'OCR_Internet_Collection_EUS26.jpg',
 'OCR_Intern

In [116]:
import os

def get_list_file(fol):
    list_dir = os.listdir(fol)
    try:
        list_dir.remove('.ipynb_checkpoints')
    except:
        pass
    return list_dir
# dict_temp = {}
# dict_temp1 = {}
# for a in list_dir:
#     try:
#         path = 'cor_tab/' + a
#         cor, len_edge = detect_cordinate(path)
#         # dict_temp[a] = {'coordinate': cor, 'edge_len': len_edge}
#         dict_temp[a] = cor
#         dict_temp1[a] = len_edge
#     except:
#         print(a)

In [292]:
# Folder definition
edge_fol = "crops/LenE/"
point_fol = "crops/Point/"
x_fol = "crops/X/"
y_fol = "crops/Y/"

list_fol = {"x" : x_fol,
            "y" : y_fol, 
            "edge": edge_fol, 
            "point" : point_fol}
dict_temp = {}

for name, fol in list_fol.items():
    dict_temp[name] = get_list_file(fol)

In [118]:
# temp = {}
# sample = 'OCR_Internet_Collection_EUS20.jpg'
# sample2 = 'OCR_Internet_Collection_EUS202.jpg'
# for key, list_f in dict_temp.items():
#     if sample in list_f:
#         temp[key] = detect_text(list_fol[key]+ sample)[0]
#     if sample2 in list_f:
#         temp[key + "2"] = detect_text(list_fol[key] + sample2)[0]

In [293]:
def extract_test(img):
    temp = {}
    name = img.split(".")[0]
    sample = name+'.jpg'
    sample2 = name+'2.jpg'
    for key, list_f in dict_temp.items():
        if sample in list_f:
            temp[key] = detect_text(list_fol[key]+ sample)[0]
        if sample2 in list_f:
            temp[key + "2"] = detect_text(list_fol[key] + sample2)[0]
    return temp

In [294]:
dict_ocr = {}
for i in list_dir:
    dict_ocr[i] = extract_test(i)
dict_ocr

{'OCR_Internet_Collection_EUS01.jpg': {'x': 'X(m)\n1208882.9700\n1208886.4065\n1208877.6392\n12088729693\n1208881.7704\n1208882.9700',
  'y': 'Y(m)\n587324.7000\n587328.1923\n587336.9708\n587332.4487\n587323.6364\n▪ 587324.7000',
  'edge': 'Canh\n4.90\n12:41\n6.50\n12.45\n1.60',
  'point': 'số biệ\ndifm\n1\n2\n3\n5'},
 'OCR_Internet_Collection_EUS02.jpg': {'x': 'X(m)\n1201339.33\n1201338.27\n1201335.85\n1201312.81\n1201314.04\n1201336.75\n1201339.33\n1201336.17',
  'y': 'Y(m)\n602976.72\n602972.55\n602973.23\n602979.65\n602983.78\n602977.44\n602976.72\n602975.29',
  'edge': 'Canh\n4.30\n2.51\n23.93\n4.30\n23.58\n2.68',
  'point': 'Số hiệu\ndiem\n1\n2345617'},
 'OCR_Internet_Collection_EUS03.jpg': {'x': 'x (m)\n1193878.560\n1193875.700\n1193876.620\n1193877.620\n1193878.680\n1193879.470\n1193878.560',
  'y': 'x (m)\n1193878.560\n1193875.700\n1193876.620\n1193877.620\n1193878.680\n1193879.470\n1193878.560',
  'y2': '31\nY (m)\n605022450\n605021.370\n605018.810\n605019.160\n605019.570.\n6

In [307]:
dict_ocr['OCR_Internet_Collection_EUS23.jpg']

{'x': '1195678.79\n1195680.02\n1195654.20\n1195652.96\n1195678.797',
 'x2': 'Toa\nX(m)\n1195682.14\n1195680.02\n1195678,79\n1195680.87\n1195682.14:\n1195678.79\n1195680.02\n1195654.20\n1195652.96\n1195678.797',
 'y': '1195678.79\n1195680.02\n1195654.20\n1195652.96\n1195678,797',
 'y2': 'Toad\nX(m)\n1195682.14\n1195680.02\n1195678.79\n1195680.87\n1195682.147\n1195678.79\n1195680.02\n1195654.20\n1195652.96\n1195678.79',
 'point': 'SỐ HIỆU\ndifm\n3\n4\n1',
 'point2': 'SỐ HIỆU\ndifm\n3'}

In [317]:
def check_point(x):
    try:
        x.index('.')
        return True
    except:
        return False
def detect_coordinateV2(img, key):
    if ('y' in dict_ocr[img][key]) or ('Y' in dict_ocr[img][key]):
        label = 'y'
    elif ('x' in dict_ocr[img][key]) or ('X' in dict_ocr[img][key]):
        label = 'x'
    else:
        label = key
    
    list_temp = dict_ocr[img][key].split('\n')

    list_temp11 = []
    for a in list_temp:
        if (" " in a) and (len(a) > 12):
            list_temp11 = list_temp11 + a.split(" ")
        else:
            list_temp11.append(re.sub(' +', '.', a))

    list_temp1 = replace_invalid(list_temp11)

    list_temp2 = []
    for a in list_temp1:
        if a[0:1].isdigit():
            # a1 = a.replace(' ','.')

            a1 = re.sub(' +', '.', a)
            a2 = a1.replace(',','.')
            list_temp2.append(a2)

    index_point = []
    for a in list_temp2:
        try:
            index_point.append(a.index('.'))
        except:
            index_point.append(-1)

    
    new_list_temp = []   
    for l in list_temp2:
        if check_point(l) == False:
            i = max(index_point)
            new_list_temp.append(l[:i]+"."+l[i:])
        else:
            new_list_temp.append(l)
    new_list_temp1 = []
    if key not in ["edge2", "edge"]:
        new_list_temp1 = [a for a in new_list_temp if len(a) > 5] 
    else:
        new_list_temp1 = new_list_temp
    if label == "x2":
        label = "x"
    if label == "y2":
        label = "y"
    if label == "edge2":
        label = "edge"
    return {label: new_list_temp1} 

In [318]:
a = detect_coordinateV2('OCR_Internet_Collection_EUS25.jpg', "edge")
a

{'edge': ['4.07', '12.66', '4.76', '12.60']}

In [319]:
# flow
name_dict = []
x_dict = []
y_dict = []
p_dict = []
for key, value in dict_ocr.items():
    name_dict.append(key)
    try:
        x = detect_coordinateV2(key, "x")
        x_dict.append(x["x"])
    except:
        try:
            x = detect_coordinateV2(key, "x2")
            x_dict.append(x["x"])
        except:
            x_dict.append([])
    try:
        y = detect_coordinateV2(key, "y")
        y_dict.append(y["y"])
    except:
        try:
            y = detect_coordinateV2(key, "y2")
            y_dict.append(y["y"])
        except:
            y_dict.append([])
    try:
        p = detect_coordinateV2(key, "edge")
        p_dict.append(p["edge"])
    except:
        p_dict.append([])

In [320]:
import pandas as pd
df = pd.DataFrame(columns = ["img", "edge_len"])
df["img"] = name_dict
df["x_coor"] = x_dict
df["y_coor"] = y_dict
df["edge_len"] = p_dict

In [321]:
df

,img,edge_len,x_coor,y_coor
0,OCR_Internet_Collection_EUS01.jpg,"[4.90, 12.41, 6.50, 12.45, 1.60]","[1208882.9700, 1208886.4065, 1208877.6392, 120...","[587324.7000, 587328.1923, 587336.9708, 587332..."
1,OCR_Internet_Collection_EUS02.jpg,"[4.30, 2.51, 23.93, 4.30, 23.58, 2.68]","[1201339.33, 1201338.27, 1201335.85, 1201312.8...","[602976.72, 602972.55, 602973.23, 602979.65, 6..."
2,OCR_Internet_Collection_EUS03.jpg,"[3.06, 2.72, 1.06, 1.14, 0.84, 2.75]","[1193878.560, 1193875.700, 1193876.620, 119387...","[605022.450, 605021.370, 605018.810, 605019.16..."
3,OCR_Internet_Collection_EUS04.jpg,"[5.00, 20.00, 3.53, 1.47, 20.06]","[1182029.662, 1182031.941, 1182014.226, 118201...","[603722.679, 603718.229, 603708.950, 603712.07..."
4,OCR_Internet_Collection_EUS05.jpg,[],"[1195682.14, 1195680.02, 1195678.79, 1195680.8...","[583704.26, 583703.73, 583708.58, 583709.10, 5..."
5,OCR_Internet_Collection_EUS06.jpg,"[4.00, 00.1, 15.54]","[1201283.78, 1201286.67, 1201277.01, 1201273.9...","[604990.09, 604987.32, 604975.40, 604978.02, 6..."
6,OCR_Internet_Collection_EUS07.jpg,"[5.00, 22.17, 5.00, 22.12]","[1225890.38, 1225886.56, 1225872.00, 1225875.8...","[582654.01, 582657.24, 582640.52, 582637.33, 5..."
7,OCR_Internet_Collection_EUS08.jpg,[],"[1194022.31, 1194016.31, 1194014.85, 1194013.0...","[226010.9, 000001.00, 601396.90, 601396.06, 60..."
8,OCR_Internet_Collection_EUS09.jpg,"[5.00, 10.54, 5.00, 10.58]","[1202944.72, 1202940.92, 1202934.00, 1202937.7...","[600718.47, 600715.21, 600723.16, 600726.44, 6..."
9,OCR_Internet_Collection_EUS10.jpg,"[5.00, 27.11, 5.00, 27.21]","[1190671.54, 1190669.45, 1190644.82, 1190646.8...","[588787.37, 588791.91, 588780.57, 588775.99, 5..."


In [163]:
list_temp = dict_ocr['OCR_Internet_Collection_EUS18.jpg']['edge'].split('\n')

list_temp11 = []
for a in list_temp:
    if (" " in a) and (len(a) > 12):
        list_temp11 = list_temp11 + a.split(" ")
    else:
        list_temp11.append(re.sub(' +', '.', a))
        
list_temp1 = replace_invalid(list_temp11)



        
list_temp2 = []
for a in list_temp1:
    if a[0:1].isdigit():
        # a1 = a.replace(' ','.')
        
        a1 = re.sub(' +', '.', a)
        a2 = a1.replace(',','.')
        list_temp2.append(a2)
list_temp2

index_point = []
for a in list_temp2:
    try:
        index_point.append(a.index('.'))
    except:
        index_point.append(-1)
max(index_point)

def check_point(x):
    try:
        x.index('.')
        return True
    except:
        return False
new_list_temp = []   
for l in list_temp2:
    if check_point(l) == False:
        i = max(index_point)
        new_list_temp.append(l[:i]+"."+l[i:])
    else:
        new_list_temp.append(l)
        
new_list_temp

['62.09', '70.87', '46.04', '21.74', '118.35', '45.24']

In [63]:
# list_t_f = detect_text("cor_tab/OCR_Internet_Collection_EUS31.jpg")
# list_a = re.findall("\d\d\d\d\d\d\d\s\d+", list_t_f[0]) + re.findall("\d\d\d\d\d\d\s\d+", list_t_f[0])
# for a in list_a:
#     list_t_f[0] = list_t_f[0].replace(a, a.replace(' ','.'))
# list_t_1 = list_t_f[0].replace(',','.')
# temp_list = list_t_1.split('\n')
# temp_list1 = [a.split(' ') for a in temp_list]
# list_t = sum(temp_list1, [])    
# list_valid1 = [a for a in list_t if check_num(a[0])]
# list_valid2 = replace_invalid(list_valid1)
# list_valid2 = replace_invalid2(list_valid2)
# list_valid2
detect_text("crops/Y/OCR_Internet_Collection_EUS08.jpg")

['2260109\n00000100\nBESECLOD\nPEDECLOD\n601396 90\n601396 06\n60139819\n2 260109',
 '2260109',
 '00000100',
 'BESECLOD',
 'PEDECLOD',
 '601396',
 '90',
 '601396',
 '06',
 '60139819',
 '2',
 '260109']

In [111]:
import pandas as pd
df = pd.DataFrame(columns = ["img", "coordinate", "edge_len"])
df["img"] = dict_temp.keys()
df["coordinate"] = dict_temp.values()
df["edge_len"] = dict_temp1.values()

In [113]:
df

,img,coordinate,edge_len
0,OCR_Internet_Collection_EUS13.jpg,"{1: (1180831.61, 592118.2), 2: (1180832.91, 59...","[1.84, 16.3, 16.21, 1.57, 17.67, 19.39]"
1,OCR_Internet_Collection_EUS19.jpg,"{1: (1195461.829, 612483.303), 2: (1195457.686...","[5.0, 22.4, 5.0, 22.4]"
2,OCR_Internet_Collection_EUS07.jpg,"{1: (1225890.38, 582654.01), 2: (1225886.56, 5...","[5.0, 22.17, 5.0, 22.12]"
3,OCR_Internet_Collection_EUS24.jpg,"{1: (1183762.26, 585974.3), 2: (1183761.85, 58...","[5.6, 15.61, 5.6, 5.6, 1.5, 5.6, 1.59]"
4,OCR_Internet_Collection_EUS31.jpg,"{1: (1193347.11, 611450.39), 2: (1193349.2, 61...","[21.0, 10.0, 21.0, 10.0]"
5,OCR_Internet_Collection_EUS14.jpg,"{1: (1185494.37, 588592.02), 2: (1185617.72, 5...","[8.89, 124.0, 33.0, 8.88]"
6,OCR_Internet_Collection_EUS17.jpg,"{1: (1197374.41, 586741.99), 2: (1197374.33, 5...","[5.0, 23.01, 5.0, 22.75]"
7,OCR_Internet_Collection_EUS28.jpg,"{1: (12345.0, 595742.58), 2: (1210870.58, 5957...","[10.87, 1.14, 43.0, 12.0, 42.79]"
8,OCR_Internet_Collection_EUS32.jpg,"{1: (1198267.83, 596965.38), 2: (1198281.43, 5...","[31.57, 30.84, 2.8, 29.68, 33.94]"
9,OCR_Internet_Collection_EUS18.jpg,"{1: (1204311.54, 599692.47), 2: (1204250.1, 59...","[87.0, 46.04, 21.74, 118.0, 35.0, 45.24]"


In [ ]:
def check_coordinate(cor):
    count = 0
    if len(cor) > 1:
        for i in range(2,len(cor)):
            if cor[1] = cor[i]:
                count = count + 1
            else:
                
    else:
        count = 0
    